In [ ]:
import pandas as pd
from influxdb_client import InfluxDBClient

In [ ]:
client = InfluxDBClient(url="https://influx.konzept.space/",
                        token="0yLbh-D7RMe1sX1iIudFel8CcqCI8sVfuRTaliUp56MgE6kub8-nSd05_EJ4zTTKt0lUzw8zcO73zL9QhC3jtA==",
                        org="api.existenz.ch",
                        debug=False)

Fetch hourly mean for flow and temperature for Thun (id 2030) and Bern, Schönau (id 2135) of the last few days.

In [ ]:
df = client.query_api().query_data_frame(
    """
from(bucket: "existenzApi")
  |> range(start: -5d)
  |> filter(fn: (r) => r["_measurement"] == "hydro")
  |> filter(fn: (r) => r["_field"] == "temperature" or r["_field"] == "flow")
  |> filter(fn: (r) => r["loc"] == "2135" or r["loc"] == "2030")
  |> aggregateWindow(every: 1h, fn: mean, createEmpty: false)
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> drop(columns: ["_start", "result", "_stop", "table", "_measurement"])
    """
)

df = df[['_time', 'loc', 'flow', 'temperature']]
df

In [ ]:
df.dtypes

In [ ]:
df.pivot(index='_time', columns='loc', values='flow').plot()

In [ ]:
df.pivot(index='_time', columns='loc', values='temperature').plot()